<a href="https://colab.research.google.com/github/IsaacQ785/Washington-Uni/blob/Assignments/Assignment/ass7-finished.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
try:
    from google.colab import drive
    drive.mount('/content/drive', force_remount=True)
    COLAB = True
    print("Note: using Google CoLab")
    %tensorflow_version 2.x
except:
    print("Note: not using Google CoLab")
    COLAB = False

# Nicely formatted time string
def hms_string(sec_elapsed):
    h = int(sec_elapsed / (60 * 60))
    m = int((sec_elapsed % (60 * 60)) / 60)
    s = sec_elapsed % 60
    return f"{h}:{m:>02}:{s:>05.2f}"

Mounted at /content/drive
Note: using Google CoLab


In [2]:
import sys

!{sys.executable} -m pip install git+https://github.com/zzh8829/yolov3-tf2.git@master

  Cloning https://github.com/zzh8829/yolov3-tf2.git (to revision master) to /tmp/pip-req-build-tfl3ds8n
  Running command git clone -q https://github.com/zzh8829/yolov3-tf2.git /tmp/pip-req-build-tfl3ds8n
  Created wheel for yolov3-tf2: filename=yolov3_tf2-0.1-cp36-none-any.whl size=8846 sha256=3c6e855c48b0680007cef2c52965723e67f0267325e80f4bf5efaf3647b1ce69
  Stored in directory: /tmp/pip-ephem-wheel-cache-xadfue52/wheels/59/1b/97/905ab51e9c0330efe8c3c518aff17de4ee91100412cd6dd553
Successfully built yolov3-tf2


In [3]:
import tensorflow as tf
import os

if COLAB:
  ROOT = '/content/drive/My Drive/projects/t81_558_dlearning/yolo'
else:
  ROOT = os.path.join(os.getcwd(),'data')

filename_darknet_weights = tf.keras.utils.get_file(
    os.path.join(ROOT,'yolov3.weights'),
    origin='https://pjreddie.com/media/files/yolov3.weights')
TINY = False

filename_convert_script = tf.keras.utils.get_file(
    os.path.join(os.getcwd(),'convert.py'),
    origin='https://raw.githubusercontent.com/zzh8829/yolov3-tf2/master/convert.py')

filename_classes = tf.keras.utils.get_file(
    os.path.join(ROOT,'coco.names'),
    origin='https://raw.githubusercontent.com/zzh8829/yolov3-tf2/master/data/coco.names')
filename_converted_weights = os.path.join(ROOT,'yolov3.tf')

8192/1277 [================================================================================================================================================================================================] - 0s 0us/step


In [4]:
import sys
!{sys.executable} "{filename_convert_script}" --weights "{filename_darknet_weights}" --output "{filename_converted_weights}"

2020-12-22 12:56:30.611351: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudnn.so.7
2020-12-22 12:56:35.628119: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.10
I1222 12:56:37.581290 139766677772160 convert.py:31] sanity check passed
I1222 12:56:39.221966 139766677772160 convert.py:34] weights saved


In [5]:
import os
os.remove(filename_convert_script)

## START BELOW ##


In [6]:
import time
from absl import app, flags, logging
from absl.flags import FLAGS
import cv2
import numpy as np
import tensorflow as tf
from yolov3_tf2.models import (YoloV3, YoloV3Tiny)
from yolov3_tf2.dataset import transform_images, load_tfrecord_dataset
from yolov3_tf2.utils import draw_outputs
import sys
from PIL import Image, ImageFile
import requests

# Flags are used to define several options for YOLO.
flags.DEFINE_string('classes', filename_classes, 'path to classes file')
flags.DEFINE_string('weights', filename_converted_weights, 'path to weights file')
flags.DEFINE_boolean('tiny', False, 'yolov3 or yolov3-tiny')
flags.DEFINE_integer('size', 416, 'resize images to')
flags.DEFINE_string('tfrecord', None, 'tfrecord instead of image')
flags.DEFINE_integer('num_classes', 80, 'number of classes in the model')
FLAGS([sys.argv[0]])

# Locate devices to run YOLO on (e.g. GPU)
physical_devices = tf.config.experimental.list_physical_devices('GPU')
if len(physical_devices) > 0:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)

In [7]:
# This assignment does not use the "Tiny version"
if FLAGS.tiny:
    yolo = YoloV3Tiny(classes=FLAGS.num_classes)
else:
    yolo = YoloV3(classes=FLAGS.num_classes)

# Load weights and classes
yolo.load_weights(FLAGS.weights).expect_partial()
print('weights loaded')

class_names = [c.strip() for c in open(FLAGS.classes).readlines()]
print('classes loaded')

weights loaded
classes loaded


In [47]:
import pandas as pd

i = 1
url = f"https://data.heatonresearch.com/data/t81-558/sidewalk/sidewalk{i}.png"
response = requests.get(url)
img_raw = tf.image.decode_image(response.content, channels=3)

# Preprocess image
img = tf.expand_dims(img_raw, 0)
img = transform_images(img, FLAGS.size)

# Desired threshold (any sub-image below this confidence level will be ignored.)
FLAGS.yolo_score_threshold = 0.1

# Recognize and report results
boxes, scores, classes, nums = yolo(img)

# Desired threshold (any sub-image below this confidence level will be ignored.)
FLAGS.yolo_score_threshold = 0.1

submit_df = pd.DataFrame()

print('detections:')
for i in range(nums[0]):
    cls = class_names[int(classes[0][i])]
    score = np.array(scores[0][i])
    box = np.array(boxes[0][i])
    print(f"\t{cls}, {score}, {box}")


detections:
	person, 0.8610939383506775, [0.6191845  0.59739566 0.6455239  0.6964505 ]
test 0
	person, 0.8247124552726746, [0.5645817 0.6453511 0.595969  0.7455805]
test 1
	person, 0.79351407289505, [0.6079407  0.69781303 0.6366278  0.7886913 ]
test 2
	person, 0.7164221405982971, [0.6633641 0.6810441 0.6887379 0.7944299]
test 3
	person, 0.6968550682067871, [0.09189358 0.33121657 0.111054   0.37928265]
test 4
	person, 0.6268397569656372, [0.11293058 0.32814595 0.1336746  0.38183174]
test 5
	person, 0.5897690057754517, [0.6063562  0.754396   0.63623744 0.84774655]
test 6
	person, 0.43654847145080566, [0.8495312 0.4620815 0.8663175 0.5087753]
test 7
	person, 0.4356961250305176, [0.43019527 0.74990773 0.47327548 0.8516606 ]
test 8
	person, 0.4281291961669922, [0.59075445 0.6240619  0.61355954 0.6932853 ]
test 9
	person, 0.39086875319480896, [0.808963  0.5757502 0.8373168 0.6760718]
test 10
	person, 0.37690871953964233, [0.5117131  0.60740215 0.5481817  0.71740025]
test 11
	person, 0.373445

In [56]:
class_clms = ['person',	'car',	'bicycle',	'motorbike',	'umbrella',	'handbag']
class_dict = {class_names[i]:i for i in range(len(class_names))}

In [122]:
submit_df = pd.DataFrame(columns=["index",'person',	'car',	'bicycle',	'motorbike',	'umbrella',	'handbag'])

display(submit_df)


,index,person,car,bicycle,motorbike,umbrella,handbag


In [124]:
FLAGS.yolo_score_threshold = 0.1

for i in range(1,11):
  url = f"https://data.heatonresearch.com/data/t81-558/sidewalk/sidewalk{i}.png"
  response = requests.get(url)
  img_raw = tf.image.decode_image(response.content, channels=3)

  # Preprocess image
  img = tf.expand_dims(img_raw, 0)
  img = transform_images(img, FLAGS.size)

  # Recognize and report results
  boxes, scores, classes, nums = yolo(img)

  class_found = np.asarray(classes[0,:])
  class_found = class_found.tolist()[:nums[0]]
  vals = [i]
  for name in class_clms:
    x = class_found.count(class_dict[name])
    vals.append(x)
  submit_df.loc[i-1] = vals
submit_df.set_index("index",inplace=True)
display(submit_df)

,person,car,bicycle,motorbike,umbrella,handbag
index,,,,,,
1,23,0,3,4,0,0
2,27,1,8,2,0,0
3,29,0,0,0,3,0
4,51,0,6,1,1,1
5,49,0,9,0,0,0
6,61,0,2,1,1,0
7,52,0,0,0,0,0
8,56,0,1,0,0,0
9,33,1,0,0,1,0
